# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dryden,49.7833,-92.7503,2.16,41,0,4.12,CA,1730415316
1,1,adamstown,-25.0660,-130.1015,21.12,82,100,12.21,PN,1730415317
2,2,kodiak,57.7900,-152.4072,3.73,87,100,8.23,US,1730415318
3,3,puerto natales,-51.7236,-72.4875,16.25,41,0,7.20,CL,1730415319
4,4,nuuk,64.1835,-51.7216,3.34,65,100,3.60,GL,1730415322


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # Set point size by humidity
    color="Humidity",
    colormap="viridis",
    frame_width=800,
    frame_height=500,
    title="City Humidity Levels"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Define criteria for ideal weather conditions
ideal_conditions = (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) & \
                   (city_data_df["Wind Speed"] < 10) & \
                   (city_data_df["Cloudiness"] == 0)

# Apply the criteria to filter the DataFrame
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,greenville,35.6127,-77.3663,22.86,73,0,1.54,US,1730415344
66,66,salalah,17.0151,54.0924,25.05,73,0,1.54,OM,1730415397
70,70,fort bragg,35.1390,-79.0060,24.85,63,0,2.57,US,1730415402
74,74,cabo san lucas,22.8909,-109.9124,29.17,65,0,4.63,MX,1730415069
79,79,san marcos,33.1434,-117.1661,22.27,53,0,5.14,US,1730415412


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
22,greenville,US,35.6127,-77.3663,73,
66,salalah,OM,17.0151,54.0924,73,
70,fort bragg,US,35.1390,-79.0060,63,
74,cabo san lucas,MX,22.8909,-109.9124,65,
79,san marcos,US,33.1434,-117.1661,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel within 10,000 meters
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1  # Limit results to the first (nearest) hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
greenville - nearest hotel: Comfort Inn & Suites
salalah - nearest hotel: Hilton Salalah Resort
fort bragg - nearest hotel: Home2 Suites by Hilton Fayetteville, NC
cabo san lucas - nearest hotel: Hotel Riu Santa Fe
san marcos - nearest hotel: Econo Lodge Inn & Suites Escondido Downtown
hamlet - nearest hotel: Quality Inn & Suites
pasni - nearest hotel: Juddi Hotel
salida - nearest hotel: La Quinta Inn
saint-pierre - nearest hotel: Villa Delisle Hotel & Spa
mangrol - nearest hotel: No hotel found
ciudad tula - nearest hotel: Hotel Mollinedo
sur - nearest hotel: Resort Sur Beach Hotel
gebeit - nearest hotel: No hotel found
les avirons - nearest hotel: Blue Margouillat
remire-montjoly - nearest hotel: Central Hôtel
manta - nearest hotel: Mantahost Hotel
timimoun - nearest hotel: فندق قورارة
noumea - nearest hotel: Hilton Noumea La Promenade Residences
tindouf - nearest hotel: محمد بوسبي
al majaridah - nearest hotel: No hotel found
porbandar - nearest hotel: Toran Tou

,City,Country,Lat,Lng,Humidity,Hotel Name
22,greenville,US,35.6127,-77.3663,73,Comfort Inn & Suites
66,salalah,OM,17.0151,54.0924,73,Hilton Salalah Resort
70,fort bragg,US,35.1390,-79.0060,63,"Home2 Suites by Hilton Fayetteville, NC"
74,cabo san lucas,MX,22.8909,-109.9124,65,Hotel Riu Santa Fe
79,san marcos,US,33.1434,-117.1661,53,Econo Lodge Inn & Suites Escondido Downtown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Configure the map plot with additional hover information
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # Size by humidity
    color="Humidity",
    colormap="viridis",
    frame_width=800,
    frame_height=500,
    title="Hotels in Ideal Locations",
    hover_cols=["City", "Country", "Hotel Name"]
).opts(
    hover_tooltips=[
        ("City", "@City"),
        ("Country", "@Country"),
        ("Hotel Name", "@Hotel Name"),
        ("Humidity", "@Humidity")
    ]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)